In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import os
from PIL import Image
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
#from linformer import Linformer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%pip install einops
%pip install wandb

     |████████████████████████████████| 2.1MB 8.3MB/s 
     |████████████████████████████████| 102kB 12.3MB/s 
     |████████████████████████████████| 133kB 33.7MB/s 
     |████████████████████████████████| 163kB 26.9MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=5a28e2e7a0bbf616eec6631d812f1015e3c9bc5152b128eea59d6412f6576e4e
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=7fd3c3822376caa64fbed65f156b6a9c91d2c4094e376c9fe2d3012738635ec8
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [4]:
%cd drive/MyDrive/Colab Notebooks/ViT

/content/drive/MyDrive/Colab Notebooks/ViT


In [9]:
from model.pit import PiT
from dataloader import set_transforms
from utils.utils import seed_everything

seed_everything(42)

In [10]:
import torch.backends.cudnn as cudnn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = torch.cuda.is_available()
cudnn.benchmark = True
print('Use CUDA:', use_cuda)

Use CUDA: True


In [11]:
config={
    "epochs": 20, 
    "batch_size": 64,
    "lr" : 3e-5,
    "gamma" : 0.7,
    "image_size" : 32,
    "patch_size" : 16,
    "num_classes" : 10,
    "dim" : 128,
    "depth" : (3,3,3),
    "heads" : 8,
    "mlp_dim" : 1024,
    "channels" : 3,
    "dropout" : 0.,
    "emb_dropout": 0.
    }

# Training settings
import wandb
wandb.init(config=config, project="Cifar10_ViT", entity='NoguNogu',name='PiT')

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [12]:
trainloader, testloader = set_transforms(
    config['image_size'],
    config['batch_size'],
    name='Cifar10')._set_transforms()

In [13]:
model = PiT(
    image_size = config['image_size'],
    patch_size = config['patch_size'],
    num_classes = config['num_classes'],
    dim = config['dim'],
    depth = config['depth'],
    heads = config['heads'],
    mlp_dim = config['mlp_dim'],
    dim_head = 64,
    dropout = config['dropout'],
    emb_dropout = config['emb_dropout']
    ).to(device)

In [14]:
from torchsummary import summary
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Unfold-1               [-1, 768, 9]               0
         Rearrange-2               [-1, 9, 768]               0
            Linear-3               [-1, 9, 128]          98,432
           Dropout-4              [-1, 10, 128]               0
         LayerNorm-5              [-1, 10, 128]             256
            Linear-6             [-1, 10, 1536]         196,608
           Softmax-7            [-1, 8, 10, 10]               0
            Linear-8              [-1, 10, 128]          65,664
           Dropout-9              [-1, 10, 128]               0
        Attention-10              [-1, 10, 128]               0
          PreNorm-11              [-1, 10, 128]               0
        LayerNorm-12              [-1, 10, 128]             256
           Linear-13             [-1, 10, 1024]         132,096
             GELU-14             [-1, 1

In [15]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=config['lr'])
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=config['gamma'])

In [16]:
# 3. Log gradients and model parameters
wandb.watch(model, log_freq=100)
for epoch in range(config['epochs']):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(trainloader):
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(trainloader)
        epoch_loss += loss / len(trainloader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in testloader:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(testloader)
            epoch_val_loss += val_loss / len(testloader)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )

    # 4. Log metrics to visualize performance
    wandb.log({
        "Epoch": epoch+1,
        "loss": epoch_loss,
        "acc": epoch_accuracy,
        "val_loss" : epoch_val_loss,
        "val_acc": epoch_val_accuracy
        })


Epoch : 1 - loss : 2.1018 - acc: 0.2063 - val_loss : 1.9990 - val_acc: 0.2478




Epoch : 2 - loss : 1.9731 - acc: 0.2665 - val_loss : 1.9284 - val_acc: 0.2877




Epoch : 3 - loss : 1.8986 - acc: 0.2986 - val_loss : 1.8499 - val_acc: 0.3260




Epoch : 4 - loss : 1.8405 - acc: 0.3219 - val_loss : 1.8010 - val_acc: 0.3407




Epoch : 5 - loss : 1.7852 - acc: 0.3485 - val_loss : 1.7400 - val_acc: 0.3641



...


Epoch : 16 - loss : 1.5587 - acc: 0.4368 - val_loss : 1.5701 - val_acc: 0.4315




Epoch : 17 - loss : 1.5442 - acc: 0.4449 - val_loss : 1.5603 - val_acc: 0.4394




Epoch : 18 - loss : 1.5363 - acc: 0.4477 - val_loss : 1.5619 - val_acc: 0.4416




Epoch : 19 - loss : 1.5231 - acc: 0.4528 - val_loss : 1.5384 - val_acc: 0.4422




Epoch : 20 - loss : 1.5139 - acc: 0.4557 - val_loss : 1.5309 - val_acc: 0.4484



In [17]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu May  6 13:23:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    40W / 250W |   1445MiB / 16280MiB |     19%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------